In [6]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [7]:
# ruta para el metodo Path
directorio = Path('C:\\Users\\sbstn\\Python\\github\\DataElite\\dataelite\\05_manipulacion_datos_tradicional\\Retos\\Soluciones\\src')

# obtiene los nombres de los csv
videos_csv = [video.name for video in directorio.glob('*.csv')]
# obtiene los nombres de los json
cat_json = [video.name for video in directorio.glob('*.json')]

In [8]:
# funcion que obtiene el json del pais correspondiente al video, extrae los ID y los titulos asociados en un dataframe y luego lo une con el df de los csv
def get_title(df, country):
    data = json.load(open(f'src/{country}_category_id.json'))
    df_temp = pd.json_normalize(data['items'], max_level=1)
    df_cat = df_temp[['id','snippet.title']]
    df_cat = df_cat.rename(columns={'snippet.title': 'category', 'id': 'category_id'})
    df_cat['category_id'] = df_cat['category_id'].astype(int)
    df_cat = df.merge(df_cat, how='left', on='category_id')
    return df_cat

In [9]:
# Concatena los dataframe de todos los csv de videos
df_videos = pd.DataFrame()
for video in videos_csv:
    df = pd.read_csv(f'src/{video[:2]}videos.csv', encoding='latin-1')
    df['country'] = video[:2]
    df = get_title(df, video[:2])
    df_videos = pd.concat([df_videos, df], ignore_index=True)
df_videos


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,category
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA,Music
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,Comedy
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA,Comedy
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,Entertainment
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA,Music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18T13:00:04.000Z,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals,US,Pets & Animals
375938,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18T01:00:06.000Z,[none],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN,US,People & Blogs
375939,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18T17:34:22.000Z,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...,US,Entertainment
375940,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17T17:00:04.000Z,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...,US,Film & Animation


In [10]:
# Transforma las 2 columnas de fecha a tipo datetime y luego sustrae ambas para calcular los días que demoro en ser trending
df_videos['trending_date'] = pd.to_datetime(df_videos['trending_date'], format='%y.%d.%m')
df_videos['publish_time'] = pd.to_datetime(df_videos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df_videos['publish_time'] = df_videos['publish_time'].dt.date.astype('datetime64[ns]')
df_videos['days_to_trend'] = (df_videos['trending_date'] - df_videos['publish_time']).dt.days
df_videos.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,category,days_to_trend
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA,Music,4
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,Comedy,1
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA,Comedy,2
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,Entertainment,2
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA,Music,5


In [11]:
# Extrae el dia de trending_date y luego crea una tabla pivot para contar la cantidad de videos de cada dia de la semana por pais
df_videos['trending_day'] = df_videos['trending_date'].dt.day_name()
trend_days_table = pd.pivot_table(df_videos,values='title', index='trending_day', columns='country', aggfunc='count')
trend_days_table

country,CA,DE,FR,GB,IN,JP,KR,MX,RU,US
trending_day,,,,,,,,,,
Friday,5787,5791,5773,5528,5289,2970,4955,5735,5785,5793
Monday,5759,5747,5775,5471,5264,2937,4892,5698,5772,5793
Saturday,5990,5993,5980,5739,5617,3022,5055,5925,5947,5993
Sunday,5794,5767,5730,5483,5356,2804,4810,5745,5732,5794
Thursday,5783,5780,5749,5489,5182,2868,4818,5709,5783,5793
Tuesday,5988,5986,5978,5674,5476,2953,5116,5943,5953,5992
Wednesday,5780,5776,5739,5532,5168,2969,4921,5696,5767,5791


In [12]:
# Convierte publish_time en una columna tipo date y luego crea una tabla pivot para contar la cantidad de videos por cada fecha
#  rellenando con 0 los valores nulos
df_videos['publish_time_date'] = df_videos['publish_time'].dt.date
publish_date_table = pd.pivot_table(df_videos,values='title', index='publish_time_date', columns='country', aggfunc='count', fill_value=0)
publish_date_table

country,CA,DE,FR,GB,IN,JP,KR,MX,RU,US
publish_time_date,,,,,,,,,,
2006-07-23,0,0,0,0,0,0,0,0,0,1
2007-03-05,0,0,0,9,0,0,0,0,0,0
2007-06-25,0,0,0,12,0,0,0,0,0,0
2007-12-03,0,0,0,16,0,0,0,0,0,0
2008-01-07,0,0,0,10,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2018-06-10,196,266,248,9,149,297,196,199,288,25
2018-06-11,192,190,198,22,197,169,139,195,194,36
2018-06-12,178,176,169,15,110,167,166,134,185,14


In [13]:
# divide la columna tags por el separador y luego cuenta la cantidad de palabras que separo para crear una nueva columna
df_videos['tags_per_video'] = df_videos['tags'].str.split(pat='|').str.len()
df_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,country,category,days_to_trend,trending_day,publish_time_date,tags_per_video
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,...,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA,Music,4,Tuesday,2017-11-10,6
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,...,False,False,False,STill got a lot of packages. Probably will las...,CA,Comedy,1,Tuesday,2017-11-13,11
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,...,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA,Comedy,2,Tuesday,2017-11-12,23
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,...,False,False,False,I know it's been a while since we did this sho...,CA,Entertainment,2,Tuesday,2017-11-12,14
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,...,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA,Music,5,Tuesday,2017-11-09,10


In [14]:
# Calcula el promedio de tags por video en los videos que fueron tendencia en 3 dias o menos
promedio_tags = df_videos[df_videos['days_to_trend'] <= 3]['tags_per_video'].mean()
print(f'el promedio de tags por videos que fueron tendencia en menos de 3 dias es {promedio_tags:.0f}')


el promedio de tags por videos que fueron tendencia en menos de 3 dias es 17


In [15]:
# Crea una tabla agrupando por pais y por categoria y luego cuenta la cantidad de videos que hay por por cada pais en cada categoria
# luego suma el total de videos por cada pais
# transforma en porcentaje del total de videos por pais, por cada categoria y le da formato de porcentaje
# Aplica la funcion pivot para mosntrar, por cada pais en cada categoria, el porcenataje de esta respecto al total del pais
df_country_cat = df_videos.groupby(['country','category']).size().reset_index(name='porcentaje')
total_videos_pais = df_country_cat.groupby('country')['porcentaje'].transform('sum')
df_country_cat['porcentaje'] = ((df_country_cat['porcentaje'] / total_videos_pais)*100).map('{:.2f}%'.format)
pv_df = df_country_cat.pivot(values='porcentaje', columns='country', index='category')
pv_df

country,CA,DE,FR,GB,IN,JP,KR,MX,RU,US
category,,,,,,,,,,
Autos & Vehicles,0.87%,2.15%,1.66%,0.37%,0.19%,1.37%,0.35%,0.63%,4.04%,0.94%
Comedy,9.25%,6.24%,10.69%,4.71%,9.21%,3.62%,6.00%,4.33%,7.82%,8.44%
Education,2.43%,2.08%,1.89%,1.18%,3.29%,0.55%,1.42%,1.32%,1.82%,4.04%
Entertainment,32.96%,37.68%,24.18%,23.50%,44.87%,30.52%,26.12%,33.55%,15.16%,24.33%
Film & Animation,5.05%,5.85%,5.31%,6.64%,4.45%,5.95%,6.42%,3.23%,7.76%,5.73%
Gaming,3.29%,3.86%,3.59%,4.61%,0.18%,5.02%,4.06%,2.47%,2.66%,2.00%
Howto & Style,4.92%,4.30%,5.81%,4.97%,2.27%,3.90%,1.63%,6.14%,5.10%,10.12%
Movies,0.01%,0.00%,0.03%,NaN,0.04%,NaN,NaN,NaN,0.00%,NaN
Music,9.14%,5.84%,9.72%,35.42%,10.36%,6.29%,5.32%,8.39%,4.83%,15.81%


In [16]:
# crea una tabla con la suma de los likes, dislikes y views por categoria
# crea la columna engagement rate y luego ordena la tabla en orden descendente
df_engagement = df_videos.groupby('country').agg({
    'likes' : 'sum',
    'dislikes' : 'sum',
    'views' : 'sum'
}).reset_index()

df_engagement['engagement_rate'] = ((df_engagement['likes'] - df_engagement['dislikes'])/df_engagement['views'])
df_engagement_sort = df_engagement.sort_values(by='engagement_rate',ascending=False)
df_engagement_sort

,country,likes,dislikes,views,engagement_rate
8,RU,506598491,60098157,9806494525,0.045531
7,MX,641627186,30223385,13849692994,0.044146
2,FR,708144090,33188528,17100897444,0.039469
1,DE,893395538,57059031,24645115205,0.033935
0,CA,1618179878,82137919,46891975069,0.032757
9,US,3041147198,151978155,96671770152,0.029886
5,JP,165406898,7528321,5377466630,0.029359
6,KR,421247912,18634999,14689152313,0.027409
4,IN,1011593670,62194142,39610961029,0.023968
3,GB,5234962944,296250384,230069198174,0.021466


In [22]:
# Agrupa por el dia de la semana y describe por la cantidad de días que toma en ser trend un video segun cada dia de la semana
df_videos.groupby('trending_day')['days_to_trend'].describe()

,count,mean,std,min,25%,50%,75%,max
trending_day,,,,,,,,
Friday,53406.0,6.904749,91.367308,0.0,1.0,1.0,3.0,3987.0
Monday,53108.0,8.135159,104.016041,0.0,1.0,1.0,3.0,4215.0
Saturday,55261.0,6.972784,92.197105,0.0,1.0,1.0,3.0,3988.0
Sunday,53015.0,7.393455,96.669751,0.0,1.0,2.0,3.0,3989.0
Thursday,52954.0,7.248574,93.515579,0.0,1.0,1.0,3.0,3986.0
Tuesday,55059.0,8.271745,104.853452,0.0,1.0,1.0,3.0,3984.0
Wednesday,53139.0,7.556691,97.944790,0.0,1.0,1.0,3.0,3985.0


Podemos notar que para la mayoria de los datos coinciden para todos los días o varian muy poco como para concluir que el día de la semana en que un video es publicado, influye en los días que le toma para ser trending.

In [23]:
# primero debemos crear el engagement rate para toda la tabla en cada video
df_videos['engagement_rate'] = ((df_videos['likes'] - df_videos['dislikes'])/df_videos['views'])


In [25]:
# Crea una pivot table con el promedio del engagement rate para la combinación de comentarios y ratings habilitados y deshabilitados
pivot_table = pd.pivot_table(df_videos, values='engagement_rate', index='comments_disabled', columns='ratings_disabled', aggfunc='mean')
print(pivot_table)

ratings_disabled      False  True 
comments_disabled                 
False              0.034477    0.0
True               0.013852    0.0


Los videos con comentarios deshabilitados tienen un mucho menor engagement rate que los que los tienen habilitados. Para los que tienen el rating deshabilitado, da 0 ya que no posee likes ni dislikes.